In [165]:
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader  # Import PyPDF2 library
from langchain.schema import Document
from langchain_community.llms import GPT4All as gpt
from gpt4all import GPT4All
from pydantic import PrivateAttr

import re
import json

In [145]:
pdf_path = 'C:/Users/utkut/Desktop/dataset/ragdata/portfolio.pdf'
json_path = 'C:/Users/utkut/Desktop/dataset/company_metadata.json'

In [146]:
def preprocess_text(text):
    text_lower = text.lower()
    text_no_punctuation = re.sub(r'[^\w\s\$\%\.\,\"\'\!\?\(\)]', '', text_lower)
    text_normalized_tabs = re.sub(r'(\t)+', '', text_no_punctuation)
    return text_lower

# Process PDF documents
reader = PdfReader(pdf_path)
documents = []

# Extract text content from each page
for page in reader.pages:
    documents.append(preprocess_text(page.extract_text()))

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")
pdf_docs = text_splitter.create_documents(documents)

# Process JSON data
with open(json_path, 'r', encoding='latin-1') as f:
    json_data = json.load(f)

json_documents = []
for entry in json_data:
    content = preprocess_text(json.dumps(entry))
    json_documents.append({'page_content': content}) 

# Combine processed data
all_docs = pdf_docs #+ [Document(page_content=doc['page_content']) for doc in json_documents]

In [147]:
# Embeddings and Qdrant setup
'''tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2", force_download=True, resume_download=False)
model = AutoModel.from_pretrained("sentence-transformers/distiluse-base-multilingual-cased-v2", force_download=True, resume_download=False)'''

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2", model_kwargs={'device': "cpu"})
qdrant = Qdrant.from_documents(
    all_docs,
    embeddings,
    location=":memory:",
    collection_name="financial_data",
    force_recreate=True
)

#location=":memory:",

c:\Users\utkut\anaconda3\envs\finy\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [148]:
query = "Portföy optimizasyonu nedir?"
found_docs = qdrant.similarity_search_with_score(query,k=3)
print(found_docs[0]) # print the first element

(Document(page_content='kritik unsuru çeşitli risk / getiri potansiyeline sahip alternatif portföyler \narasından  optimal  portföyün belirlenmesi oluşturmaktadır. bu amaçla da hem \nliteratürde hem de uygulamada port föy optimizasyon yöntemlerinden \nyararlanılmaktadır.  \nliteratürde öneminden dolayı oldukça farklı portföy optimizasyon \nyöntemleri nin kullanıldığı görülmektedir. bu yöntemlere örnek olarak genetik \nalgoritmaya dayalı portföy optimizasyon yöntemi, parçacık sürü  optimizasyon \nyöntemi, çok amaçlı genetik algoritma yöntemi,  makine öğrenmesi ve derin \nöğrenme algoritmaları,  hedef programlama yöntemi, bulanık ortalama mutlak \nsapma yöntemi, black -litterman yöntemi, yapay sinir ağı yöntemi ve stokastik  \nprogramlamaya daya lı yöntemler verilebilir (örneğin bakınız:  çelengi, eğrioğlu \nve çorba, 2015 ; fernandez ve gomez , 2007 ; gökmen, 2009 ; klega, 2013 ; ma, \nhan ve wang, 2021 ; özdemir, 2011 ; solatikia, kiliç ve weber, 2014 ; yakut ve \nçankal, 2016 )', meta

In [149]:
def format_docs(query):
    found_docs = qdrant.similarity_search_with_score(query,k=1)
    return "\n\n".join(doc[0].page_content for doc in found_docs)

In [161]:
# Step 1: Define the Runnable interface
class Runnable:
    def run(self, input):
        raise NotImplementedError("Subclasses must implement this method")

# Step 2: Implement Runnable in GPT4All
class GPT4All(Runnable):
    
    def __init__(self,model_name,n_threads,allow_download):
        self.model_name = model_name
        self.n_threads = n_threads
        self.allow_download = allow_download


        


class StreamingStdOutCallbackHandler:
    def __init__(self, callbacks=None):  # Fixed typo here
        self.callbacks = callbacks if callbacks is not None else []
        
    def add_callback(self, callback):
        self.callbacks.append(callback)
        
# Step 3 & 4: Modify LLMChain to use Runnable instances
class LLMChain:
    def __init__(self, prompt, llm, verbose=False):
        if not isinstance(llm, Runnable):
            raise TypeError("llm must be an instance of Runnable")
        self.prompt = prompt
        self.llm = llm
        self.verbose = verbose

    def invoke(self, input):
        # Use the run method of the llm (which is guaranteed to be a Runnable)
        return self.llm.run(input)
    




In [125]:
'''model = GPT4All(model_name="mistral-7b-instruct-v0.1.Q4_0.gguf", n_threads=4, allow_download=True)
#SADECE MODELİ İNDİRİYOR'''





'model = GPT4All(model_name="mistral-7b-instruct-v0.1.Q4_0.gguf", n_threads=4, allow_download=True)\n#SADECE MODELİ İNDİRİYOR'

In [166]:
llm = gpt(
            model="mistral-7b-instruct-v0.1.Q4_0.gguf",
            max_tokens=300,
            n_threads = 4,
            temp=0.3,
            top_p=0.2,
            top_k=40,
            n_batch=8,
            seed=100,
            allow_download=True,
            verbose=True)






In [167]:
#template = '''[INST]: Sen kullanıcıların şirketler ve piyasalar hakkında bilgi alabildiği, portföylerini yorumlayabilen uzman bir finansal asistansın. Aşağıda soruların sorulacağı taslak mevcuttur, değerli görüşlerinizi bu taslağa göre belirtmen istenmektedir.[\INST]\n
#Context: {context}.\n
#Question: {question}\n
#Answer: '''

template = """
### [INST] 
Instruction: Sen kullanıcıların şirketler ve piyasalar hakkında bilgi alabildiği, portföylerini yorumlayabilen uzman 
bir finansal asistansın. Aşağıda soruların sorulacağı taslak mevcuttur, değerli görüşlerinizi bu taslağa göre belirtmen istenmektedir:

{context}

### QUESTION:
{question} 

[/INST]
 """

rag_prompt = PromptTemplate(template=template, input_variables=["context", "question"])

callbacks = [StreamingStdOutCallbackHandler()]

#retriever = qdrant.as_retriever()

llm_chain = LLMChain(prompt=rag_prompt, llm=llm, verbose = True)






In [168]:
query = "Markowitz yöntemi nedir?"
resp = llm_chain.invoke(
    input={"question":query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:

### [INST] 
Instruction: Sen kullanıcıların şirketler ve piyasalar hakkında bilgi alabildiği, portföylerini yorumlayabilen uzman 
bir finansal asistansın. Aşağıda soruların sorulacağı taslak mevcuttur, değerli görüşlerinizi bu taslağa göre belirtmen istenmektedir:

yeterli derinliği olan sermaye piyasaları ile daha uyumlu  olmasıdır2. çalışmada 
ayrıca literatürdeki temel portföy optimizasyon yöntemlerde n biri olması 
nedeniyle markowitz (1952) ortalama varyans yöntemine de yer verilmiştir.  
çalışmanın literatüre çe şitli açılardan katkı sağladığı düşünülmektedir. 
öncelikle daha önce de ifade edildiği gibi ulusal yazında portföy optimizasyon 
konusunun oldukça ilgi  görmesine rağmen, o mega  rasyosu  ile cvar ve mdd 
yöntemlerinin henüz pek kullanılmadığı görülmektedir. i̇kinci olarak, bilindiği 
gibi, ilgili dört portfö y optimizasyon yönteminin ortak bir noktası 
bulunmaktadır. o da şudur ki ; ilgili dört portföy optimiz